In [1]:
from copy import copy
import config
from tqdm import tqdm
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt

In [2]:
Train_label = np.load(config.train_label_path)
Train_seq = np.load(config.train_sequence_path)

In [32]:
pies = np.sum(Train_label, axis=1)/Train_label.shape[1]
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.pie(pies[i],labels=[0,1,2,3])
plt.show()

KeyboardInterrupt: 

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/tkinter/__init__.py", line 746, in callit
    func(*args)
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel/eventloops.py", line 231, in on_timer
    self.func()
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 298, in do_one_iteration
    stream.flush(zmq.POLLIN, 1)
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 357, in flush
    self._handle_recv()
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 480, in _handle_recv
    self._run_callback(callback, msg)
  File "/Users/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/zmq/eventloop/zmqstream.p

In [4]:
num_train = Train_label.shape[1]
label_distribuition = np.sum(Train_label, axis=1)/105000
num_classes = label_distribuition.shape[0]
num_subclasses = label_distribuition.shape[1]

In [73]:
vote_code_count = [-12, -1, 0, 1, 5]
vote_code_rate = [0.6, 0.4, 0.15, 0.05, 0.]
label_votes = np.zeros(label_distribuition.shape, dtype=float)

for row in range(num_classes):
    for col in range(num_subclasses):
        label_votes[row][col] = vote_code_count[5 - np.sum((label_distribuition[row][col] > vote_code_rate), dtype=int)]
        # print(label_votes)

In [74]:
class_group = [[0,1,2],
              [3,4,5,6],
              [7,8,9],
              [10,11,12,13],
              [14,15,16,17],
              [18,19]]
num_groups = len(class_group)

label_replicate_counts = np.ones(shape=(num_groups, num_train))
_Train_label = np.vstack(([np.expand_dims(copy(Train_label), axis=0)] * num_groups))

for g in tqdm(range(num_groups)):
    for i in range(num_train):
        tmp_count = np.sum(np.sum(np.multiply(Train_label[class_group[g],i,:], label_votes[class_group[g],:]), axis=1), axis=0)
        if tmp_count > 0:
            label_replicate_counts[g][i] = 40#tmp_count
    #     label_replicate_counts[i] = np.sum(np.sum(np.multiply(Train_label[:,i,:], label_votes), axis=1), axis=0)
        _Train_label[g,:,i,:] *= label_replicate_counts[g][i]

100%|██████████| 6/6 [00:15<00:00,  2.57s/it]


In [78]:
sum(label_replicate_counts[1])

118689.0

In [80]:
for g in range(num_groups):
    print(np.sum(_Train_label[g][class_group[g]], axis=1)/sum(label_replicate_counts[g]))

[[0.08001979 0.25323328 0.12252404 0.5442229 ]
 [0.06563549 0.26679507 0.10797925 0.5595902 ]
 [0.12893713 0.15574637 0.17628963 0.53902686]]
[[0.06747887 0.11290853 0.03804902 0.7815636 ]
 [0.12269039 0.4248077  0.08170934 0.37079257]
 [0.03494005 0.10618507 0.03086217 0.8280127 ]
 [0.03581629 0.14893545 0.067917   0.74733126]]
[[0.17528765 0.42023125 0.19294417 0.21153694]
 [0.16197969 0.48885787 0.12287366 0.22628878]
 [0.38219967 0.38878736 0.04784264 0.18117033]]
[[0.4419924  0.32709417 0.07466271 0.15625075]
 [0.40961036 0.27133515 0.12372776 0.19532673]
 [0.45849457 0.21030767 0.15198226 0.1792155 ]
 [0.376981   0.3093314  0.13657047 0.17711715]]
[[0.1703708  0.2524839  0.13653682 0.44060847]
 [0.52419263 0.38497898 0.05602983 0.03479859]
 [0.23397288 0.28965062 0.10994232 0.3664342 ]
 [0.16100918 0.28713295 0.14301907 0.40883878]]
[[0.27951518 0.18600367 0.46552148 0.06895966]
 [0.2390122  0.15134864 0.4355013  0.17413788]]


In [31]:
pies = (np.sum(_Train_label, axis=1)/np.sum(label_replicate_counts)
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.pie(pies[i],labels=[0,1,2,3])
plt.show()

KeyboardInterrupt: 

In [107]:
sum(label_replicate_counts[np.nonzero(np.array((label_replicate_counts>0), dtype=int))])

392912.0

In [108]:
replicates_Train_label = copy(Train_label)
replicates_Train_seq = copy(Train_seq)

for i in tqdm(range(num_train)):
    num_replicate = int(label_replicate_counts[i])
    if num_replicate > 0:
        replicates_Train_label = np.concatenate((replicates_Train_label, 
                                                 [[t]*num_replicate for t in Train_label[:,i,:]]), axis=1)
#         replicates_Train_seq = np.concatenate((replicates_Train_seq, [Train_seq[i,:]] * num_replicate),axis=0)


 12%|█▏        | 12489/105000 [00:33<04:38, 332.16it/s]


 22%|██▏       | 22615/105000 [01:08<14:20, 95.72it/s]


 25%|██▍       | 25867/105000 [01:52<15:56, 82.73it/s]


 28%|██▊       | 28926/105000 [02:39<12:52, 98.45it/s]


 31%|███       | 32054/105000 [03:28<28:13, 43.08it/s]


...

 95%|█████████▍| 99588/105000 [2:11:40<03:22, 26.78it/s]


 96%|█████████▋| 101207/105000 [2:12:28<02:10, 29.09it/s]


 98%|█████████▊| 102886/105000 [2:13:14<00:37, 55.94it/s]


100%|█████████▉| 104543/105000 [2:14:01<00:16, 27.25it/s]


100%|██████████| 105000/105000 [2:14:13<00:00, 13.04it/s]

In [109]:
pies = np.sum(replicates_Train_label, axis=1)/replicates_Train_label.shape[1]
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.pie(pies[i])

In [110]:
plt.show()

KeyboardInterrupt: 

In [135]:
from Preprocessing import Preprocessor
preprocessor = Preprocessor()
preprocessor.removeData(trainData=Train_seq, trainLabel=Train_label)



  0%|          | 0/300 [00:00<?, ?it/s]



IndexError: too many indices for array

In [122]:
rm_Idxs20 = [[3],[3],[3],[3],[1,3],[3],[3],[3],[3],[3],[1,3],[3],[3],[1,3],[1,3],[0,1],[1,3],[3],[1],[1,3]]

trans_matrix = []
for i in range(len(rm_Idxs20)):
    tmp_list = [0]*4
    for j in range(len(rm_Idxs20[i])):
        tmp_list[rm_Idxs20[i][j]] = 1
    trans_matrix.append(tmp_list)

from tqdm import tqdm
Idxs_candidate_to_remove = []
for i in tqdm(range(num_train)):
    if np.sum(np.sum(np.multiply(Train_label[:,i,:],trans_matrix), axis=0)) == 20:
        Idxs_candidate_to_remove.append(i)



  0%|          | 0/105000 [00:00<?, ?it/s]

  2%|▏         | 2337/105000 [00:00<00:04, 23365.34it/s]

  5%|▌         | 5665/105000 [00:00<00:03, 25657.49it/s]

  9%|▊         | 9026/105000 [00:00<00:03, 27617.29it/s]

 12%|█▏        | 12301/105000 [00:00<00:03, 28979.39it/s]

 15%|█▍        | 15370/105000 [00:00<00:03, 29469.99it/s]

 17%|█▋        | 18071/105000 [00:00<00:03, 28684.94it/s]

 20%|█▉        | 20711/105000 [00:00<00:03, 27381.89it/s]

 22%|██▏       | 23429/105000 [00:00<00:02, 27320.76it/s]

 25%|██▍       | 26107/105000 [00:00<00:02, 27154.69it/s]

 27%|██▋       | 28770/105000 [00:01<00:02, 26994.54it/s]

 30%|██▉       | 31432/105000 [00:01<00:02, 26880.10it/s]

 32%|███▏      | 34111/105000 [00:01<00:02, 26852.36it/s]

 35%|███▌      | 36829/105000 [00:01<00:02, 26949.73it/s]

 38%|███▊      | 40000/105000 [00:01<00:02, 28219.90it/s]

 41%|████▏     | 43349/105000 [00:01<00:02, 29616.75it/s]

 44%|████▍     | 46519/105000 [00:01<00:01, 30211.70it/s]

 47%|████▋   